# Agents and Tools

#### ChatGPT Functions

In [26]:
import sqlite3

conn = sqlite3.connect('db.sqlite')

In [27]:
def run_query(query:str):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        return cursor.fetchall()
    except sqlite3.OperationalError as err:
        return f"The following error occurred: {str(err)}"
    
def describe_tables(tables_names):
    c = conn.cursor()
    tables = ', '.join("'"+ table + "'" for table in tables_names )
    rows = c.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name IN ({tables});").fetchall()
    return '\n'.join([str(row[0]) for row in rows if row[0] is not None])


In [28]:
from langchain.tools import Tool


run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="""
    run a query on the sqlite database
    """,
    func=run_query,
)

describe_tables_tool = Tool.from_function(
    name="describe_tables",
    description="""
    Given a list of tables name, return the tables that are present in the database
    """,
    func=describe_tables,
)

In [29]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor
from dotenv import load_dotenv
from langchain.schema import SystemMessage
load_dotenv()


True

In [30]:
def list_list_tables():
    c = conn.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    rows = c.fetchall() 
    return "\n".join(row[0] for row in rows if row[0] is not None)


In [31]:
chat = ChatOpenAI(
    model='gpt-3.5-turbo',
)

tables = list_list_tables()

chat_prompt = ChatPromptTemplate(
    messages =[
        SystemMessage(
            content="""
            You are an AI that has access to SQLite database.\n
            The database contains the following tables:\n
            {tables}
            """
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [32]:
tools = [run_query_tool, describe_tables_tool]

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=chat_prompt,
    tools=tools,
)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools,
)


In [33]:
agent_executor("How many users are in the database?")



> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `SELECT COUNT(*) FROM users`


[(2000,)]There are 2000 users in the database.

> Finished chain.


{'input': 'How many users are in the database?',
 'output': 'There are 2000 users in the database.'}

In [34]:
agent_executor("How many users provided shipping address?")



> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `SELECT COUNT(DISTINCT user_id) FROM users WHERE shipping_address IS NOT NULL`


The following error occurred: no such column: user_id
Invoking: `run_sqlite_query` with `SELECT COUNT(DISTINCT id) FROM users WHERE shipping_address IS NOT NULL`
responded: It seems like the column name for user ID is not 'user_id' in the database. Let me check the correct column name and run the query again.

The following error occurred: no such column: shipping_address
Invoking: `run_sqlite_query` with `SELECT COUNT(DISTINCT user_id) FROM users WHERE shipping_address <> ''`
responded: It appears that the column name for shipping address is also not 'shipping_address' in the database. Let me find the correct column name for shipping address and run the query again.

The following error occurred: no such column: user_id
Invoking: `run_sqlite_query` with `SELECT COUNT(DISTINCT id) FROM users WHERE shipping_address IS NOT NULL`
respo

{'input': 'How many users provided shipping address?',
 'output': 'Agent stopped due to iteration limit or time limit.'}